In [86]:
%matplotlib

import serial
import numpy as np
from matplotlib.patches import Circle, Wedge, Polygon
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt
import time
import threading
import queue
from collections import OrderedDict
import math

GPS_MULT = 10000000.0
M_PI = 3.14159265358979323846

def gps_destination(lat1, lon1, angle, distance_km):
    angle = np.radians(angle);
    dx = math.sin(angle) * distance_km;
    dy = math.cos(angle) * distance_km;

    cos = math.cos(lat1 * M_PI / 180.);
    cos2 = 2. * cos * cos - 1.;
    cos3 = 2. * cos * cos2 - cos;
    cos4 = 2. * cos * cos3 - cos2;
    cos5 = 2. * cos * cos4 - cos3;

    # multipliers for converting longitude and latitude
    # degrees into distance (http://1.usa.gov/1Wb1bv7)
    kx = (111.41513 * cos - 0.09455 * cos3 + 0.00012 * cos5);
    ky = (111.13209 - 0.56605 * cos2 + 0.0012 * cos4);

    lon2 = lon1 + dx / kx;
    lat2 = lat1 + dy / ky;
    
    return  lon2, lat2


patches = []

fig, ax = plt.subplots()
d_line, = plt.plot([0], [0], 'r-x')
l_line, = plt.plot([0], [0], 'b-x')
n_line, = plt.plot([0], [0], 'k*')
plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
plt.axis('equal')
fig.canvas.set_window_title("SkyDrop task debug")
plt.show()


def redraw(circles, lines, last_lines, pilot, info, nav):
    global patches
    global d_line
           
    for p in patches:
        p.remove()
    
    xmin = 9999999999999999
    ymin = 9999999999999999
    xmax = -9999999999999999
    ymax = -9999999999999999
    
    patches = []
    p = []
    i = 0
    for i in circles:
        c = circles[i]
        
        pts = []
        
        for angle in range(0, 360):
            pts.append(gps_destination(c['y'], c['x'], angle, (c['r'] / 1000.0)))
            
        circle = Polygon(pts, alpha=0.2)
        #circle = Circle((c['x'], c['y']), deg, alpha=0.2, ls='-', lw=1)
            
        patches.append(ax.add_patch(circle))
        
        xmin = min(c['x'], xmin)
        ymin = min(c['y'], ymin)
        xmax = max(c['x'], xmax)
        ymax = max(c['y'], ymax)

    
    if pilot:
        circle = Circle((pilot['x'], pilot['y']), 0.001, lw=2)
           
        patches.append(ax.add_patch(circle))
     
    
    
    linex = []
    liney = []
    for i in lines:
        l = lines[i]
        
        lx = l['x']
        ly = l['y']
        if pilot:
            if i < pilot["w"]:
                if i < pilot["w"] - 1:
                    continue
                lx = pilot['x']
                ly = pilot['y']
            
        linex.append(lx)
        liney.append(ly)
        
        xmin = min(l['x'], xmin)
        ymin = min(l['y'], ymin)
        xmax = max(l['x'], xmax)
        ymax = max(l['y'], ymax)        

    d_line.set_data(linex, liney)

    linex = []
    liney = []    
    
    for i in last_lines:
        l = last_lines[i]
        
        lx = l['x']
        ly = l['y']
        
        if pilot:
            if i < pilot["w"]:
                if i < pilot["w"] - 1:
                    continue
                lx = pilot['x']
                ly = pilot['y']
                
        linex.append(lx)
        liney.append(ly)
        
        xmin = min(l['x'], xmin)
        ymin = min(l['y'], ymin)
        xmax = max(l['x'], xmax)
        ymax = max(l['y'], ymax) 
        
    l_line.set_data(linex, liney)

    
    if nav:
        n_line.set_data(nav['x'], nav['y'])
   
    pad = 0.1
    ax.set_xlim(xmin - pad, xmax + pad)
    ax.set_ylim(ymin - pad, ymax + pad)
    ax.set_title(info)
    fig.canvas.draw()

def read_port():
    global done
    
    circles = OrderedDict()
    lines = OrderedDict()
    last_lines = OrderedDict()
    pilot = False
    info = "no data"
    nav = False
    
    s = serial.Serial("/dev/ttyUSB0", 921600)
    while s.inWaiting() > 0:
        s.read()


    
    done = False
    
    while (not done):
        while s.inWaiting() > 0:
            try:
                line = s.readline().decode("ascii") 
            except UnicodeDecodeError:
                continue

            print(line, end = '')
            if (line[0] in ["w", "c", "r", "p", "i", "l", "n"]):
                data = line.split()
                
                if len(data) < 1:
                    continue
                
                try:
                    if data[0] == "r":
                        circles = OrderedDict()
                        lines = OrderedDict()
                        #last_lines = OrderedDict()
                        #pilot = False    
                        info = "reset"
                        
                    if data[0] == "i":
                        info = line[2:][:-1]                          
                        
                    if data[0] == "c":
                        if len(data) < 4:
                            continue
                        
                        circle = {}
                        circle["y"] = int(data[2]) / GPS_MULT
                        circle["x"] = int(data[3]) / GPS_MULT
                        circle["r"] = int(data[4])
                        i = int(data[1])                        
                        circles[i] = circle

                        line = {}
                        line["y"] = int(data[2]) / GPS_MULT
                        line["x"] = int(data[3]) / GPS_MULT
                        lines[i] = line                    

                    if data[0] == "w":
                        if len(data) < 4:
                            continue
                            
                        line = {}
                        line["y"] = int(data[2]) / GPS_MULT
                        line["x"] = int(data[3]) / GPS_MULT
                        i = int(data[1])
                        lines[i] = line
                        
                    if data[0] == "l":
                        if len(data) < 3:
                            continue
                            
                        line = {}
                        line["y"] = int(data[2]) / GPS_MULT
                        line["x"] = int(data[3]) / GPS_MULT
                        i = int(data[1])
                        last_lines[i] = line                        
                        
                    if data[0] == "p":
                        if len(data) < 4:
                            continue    
                            
                        pilot = {}
                        pilot["w"] = int(data[1])
                        pilot["y"] = int(data[2]) / GPS_MULT
                        pilot["x"] = int(data[3]) / GPS_MULT
                        
                    if data[0] == "n":
                        if len(data) < 3:
                            continue    
                            
                        nav = {}
                        nav["y"] = int(data[1]) / GPS_MULT
                        nav["x"] = int(data[2]) / GPS_MULT                    
                except:
                    pass

        redraw(circles, lines, last_lines, pilot, info, nav)
        plt.pause(0.05) 
        
    s.close()
    print("Closed port")

def handle_close(evt):
    print('Closed Figure!')
    global done
    done = True

fig.canvas.mpl_connect('close_event', handle_close)
            
read_port()    

Using matplotlib backend: TkAgg
w 4 484199872 179113296 216.00 40533
i 2 2
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484388480 174862736 190.00 66916
Loading from SD 3
w 2 481507328 178315568 345.00 47905
Loading from SD 4
w 3 482106368 178506336 202.00 30516
Loading from SD 5
w 4 484189600 179137024 210.00 40642
i 2 3
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484385024 174904864 184.00 67089
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 2 4
Loading from SD 0
Loading from SD 1
Loading from SD 2
Loading from SD 3
Loading from SD 4
Loading from SD 5
---NOT IMPROVED
i 3 0
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484384960 174906272 183.80 67089
Loading from SD 3
w 2 481507936 178318960 345.30 47549
Loading from SD 4
w 3 482105184 178510704 201.80 30426
Loading from SD 5
w 4 484189312 179137840 209.80 40648
i 3 1
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484384832 174909120 183.40 67099
Loading from SD 3
w 2 4815085

Loading from SD 2
w 1 484400640 174794432 200.00 66927
Loading from SD 3
w 2 481489952 178237824 338.00 48501
Loading from SD 4
w 3 482125120 178441616 205.00 30736
Loading from SD 5
w 4 484199872 179113296 216.00 40533
i 2 2
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484388480 174862736 190.00 66916
Loading from SD 3
w 2 481507328 178315568 345.00 47905
Loading from SD 4
w 3 482106368 178506336 202.00 30516
Loading from SD 5
w 4 484189600 179137024 210.00 40642
i 2 3
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484385024 174904864 184.00 67089
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 2 4
Loading from SD 0
Loading from SD 1
Loading from SD 2
Loading from SD 3
Loading from SD 4
Loading from SD 5
---NOT IMPROVED
i 3 0
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484384960 174906272 183.80 67089
Loading from SD 3
w 2 481507936 178318960 345.30 47549
Loading from SD 4
w 3 482105184 178510704 201.80 30426
Loading from SD 5
w 4 48418931

Loading from SD 4
w 3 482102688 178520112 201.37 30333
Loading from SD 5
w 4 484188256 179140512 209.15 40662
i 4 1
Loading from SD 0
Loading from SD 1
Loading from SD 2
Loading from SD 3
Loading from SD 4
Loading from SD 5
w 4 484188224 179140656 209.11 40662
i 4 2
Loading from SD 0
Loading from SD 1
Loading from SD 2
Loading from SD 3
Loading from SD 4
w 3 482102464 178520992 201.33 30332
Loading from SD 5
i 4 3
Loading from SD 0
Loading from SD 1
Loading from SD 2
Loading from SD 3
Loading from SD 4
Loading from SD 5
---NOT IMPROVED
i 5 0
--waypoint_task_optimise_finalise()
--waypoint_optimise_duration 14549

--waypoint_optimise_start)
r
l 0 484895566 174932600
p 1 484710583 174945816
c 0 481466333 171004333 500
l 1 484923008 174966416
c 1 484654166 174933166 3000
l 2 481512160 178344800
c 2 481073000 178489333 5000
l 3 482102464 178520992
c 3 482940166 179011166 10000
l 4 484188224 179140656
c 4 484345333 179272166 2000
l 5 485692000 179574333
c 5 485692000 179574333 1000
Loading f

Loading from SD 4
w 3 482102848 178519440 201.40 30356
Loading from SD 5
w 4 484188352 179140304 209.20 40661
i 3 2
Loading from SD 0
Loading from SD 1
Loading from SD 2
Loading from SD 3
Loading from SD 4
Loading from SD 5
---NOT IMPROVED
i 4 0
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484399616 175067584 160.65 41636
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 4 1
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484399648 175067712 160.63 41636
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 4 2
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484399712 175068032 160.58 41636
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 4 3
Loading from SD 0
Loading from SD 1
Loading from SD 2
Loading from SD 3
Loading from SD 4
Loading from SD 5
---NOT IMPROVED
i 5 0
--waypoint_task_optimise_finalise()
--waypoint_optimise_duration 15375

--waypoint_optimise_start)
r
l 0 484499183 174945816
p 1 484512400 174879750
c 0 481466333 17100433

Loading from SD 2
w 1 484391328 175024624 166.97 42048
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 4 5
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484391360 175024912 166.93 42048
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 4 6
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484391392 175025184 166.89 42048
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 4 7
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484391456 175025520 166.84 42048
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 4 8
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484391520 175025856 166.79 42048
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 4 9
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484391552 175026144 166.75 42048
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 4 10
Loading from SD 0
Loading from SD 1
Loading from SD 2
Loading from SD 3
Loading from SD 4
Loading from SD 5
---NOT IMPROVED
i 5 0

...

i 4 2
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484384384 174928080 180.72 84206
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 4 3
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484384384 174928288 180.69 84206
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 4 4
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484384384 174928496 180.66 84206
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 4 5
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484384384 174928704 180.63 84206
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 4 6
Loading from SD 0
Loading from SD 1
Loading from SD 2
Loading from SD 3
Loading from SD 4
Loading from SD 5
---NOT IMPROVED
i 5 0
--waypoint_task_optimise_finalise()
--waypoint_optimise_duration 17967

--waypoint_optimise_start)
r
l 0 481466333 171004333
c 0 481466333 171004333 500
l 1 484384384 174928704
c 1 484654166 174933166 3000
l 2 481509248 178326448
c 2 481073000 178489333 5000
l

--waypoint_optimise_duration 18064

--waypoint_optimise_start)
r
l 0 481466333 171004333
c 0 481466333 171004333 500
l 1 484384384 174928704
c 1 484654166 174933166 3000
l 2 481509248 178326448
c 2 481073000 178489333 5000
l 3 482105184 178510704
c 3 482940166 179011166 10000
l 4 484188640 179139488
c 4 484345333 179272166 2000
l 5 485692000 179574333
c 5 485692000 179574333 1000
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484447488 174672416 220.00 42838
Loading from SD 3
w 2 481417472 178057760 320.00 41980
Loading from SD 4
w 3 482095104 178550240 200.00 8373
Loading from SD 5
w 4 484176320 179179728 200.00 23605
i 1 0
Loading from SD 0
Loading from SD 1
Loading from SD 2
Loading from SD 3
Loading from SD 4
Loading from SD 5
---NOT IMPROVED
i 2 0
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484420512 174730336 210.00 84366
Loading from SD 3
w 2 481458432 178143536 329.00 49195
Loading from SD 4
w 3 482138912 178399360 207.00 31178
Loading from SD 5
w 4 484

Loading from SD 5
i 4 3
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484384384 174928288 180.69 84206
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 4 4
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484384384 174928496 180.66 84206
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 4 5
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484384384 174928704 180.63 84206
Loading from SD 3
Loading from SD 4
Loading from SD 5
i 4 6
Loading from SD 0
Loading from SD 1
Loading from SD 2
Loading from SD 3
Loading from SD 4
Loading from SD 5
---NOT IMPROVED
i 5 0
--waypoint_task_optimise_finalise()
--waypoint_optimise_duration 19518

--waypoint_optimise_start)
r
l 0 481466333 171004333
c 0 481466333 171004333 500
l 1 484384384 174928704
c 1 484654166 174933166 3000
l 2 481509248 178326448
c 2 481073000 178489333 5000
l 3 482105184 178510704
c 3 482940166 179011166 10000
l 4 484188640 179139488
c 4 484345333 179272166 2000
l 5 485692000 179574333
c 5

l 3 482105184 178510704
c 3 482940166 179011166 10000
l 4 484188640 179139488
c 4 484345333 179272166 2000
l 5 485692000 179574333
c 5 485692000 179574333 1000
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484447488 174672416 220.00 42838
Loading from SD 3
w 2 481417472 178057760 320.00 41980
Loading from SD 4
w 3 482095104 178550240 200.00 8373
Loading from SD 5
w 4 484176320 179179728 200.00 23605
i 1 0
Loading from SD 0
Loading from SD 1
Loading from SD 2
Loading from SD 3
Loading from SD 4
Loading from SD 5
---NOT IMPROVED
i 2 0
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484420512 174730336 210.00 84366
Loading from SD 3
w 2 481458432 178143536 329.00 49195
Loading from SD 4
w 3 482138912 178399360 207.00 31178
Loading from SD 5
w 4 484188032 179141120 209.00 40461
i 2 1
Loading from SD 0
Loading from SD 1
Loading from SD 2
w 1 484400640 174794432 200.00 84059
Loading from SD 3
w 2 481489952 178237824 338.00 48501
Loading from SD 4
w 3 482125120 178441616

TclError: can't invoke "update" command: application has been destroyed